In [99]:
import requests
from tqdm import tqdm_notebook
import pickle
import pandas as pd
from multiprocessing import Queue
from threading import Thread
import urllib
from time import sleep

In [100]:
vac_url = 'https://api.hh.ru/vacancies/{}'

# Давайте выкачаем немного вакансий
# Они полностью открыты через API (если их не закрыл работодатель)

vac = []
test = []
number = 23244255

Здесь парсинг проходит по одному линку в цепочке. Вместо 1000 можно указать желаемое кол-во вакансий в датасете

In [78]:
start_time = time.time()
for i in tqdm_notebook(range(number,number-1000, -1)):
    test.append(requests.get(vac_url.format(i)).json())
print("--- %s seconds ---" % (time.time() - start_time))

A Jupyter Widget


--- 134.0461757183075 seconds ---


In [129]:
def worker(url):
    vac.append(requests.get(url).json())
    return 

In [122]:
threadlist = []

In [191]:
q = Queue()
for i in tqdm_notebook(range(number,number-1000,-1)):
    q.put((vac_url.format(i)))

A Jupyter Widget

In [192]:
import time
threads = []
start_time = time.time()
for i in range(10):
    while not q.empty():
        u = q.get()
        sleep(0.027)
        t = Thread( target=worker, args = (u,))
        threads.append(t)
        t.start()
    t.join()
print("--- %s seconds ---" % (time.time() - start_time))

--- 35.34149694442749 seconds ---


In [200]:
df = pd.DataFrame(vac)
def salary_currency(x):
    return x['currency'] if type(x)==dict else None

def salary_from(x):
    return x['from'] if type(x)==dict else None

def salary_gross(x):
    return x['gross'] if type(x)==dict else None

def salary_to(x):
    return x['to'] if type(x)==dict else None

In [201]:
df['salary_currency'] = df['salary'].map(salary_currency)
df['salary_from'] = df['salary'].map(salary_from)
df['salary_gross'] = df['salary'].map(salary_gross)
df['salary_to'] = df['salary'].map(salary_to)

In [203]:
def profarea_id(x):
    return x[0]['profarea_id'] if type(x)==list else None

df['profarea_id'] = df['specializations'].map(profarea_id)

In [204]:
def area_id(x):
    return x['id'] if type(x)==dict else None

df['area_id'] = df['area'].map(area_id)

del df['area']

In [205]:
def type_id(x):
    return x['id'] if type(x)==dict else None

df['type_id'] = df['type'].map(type_id)

del df['type']

In [206]:
def billing_type_id(x):
    return x['id'] if type(x)==dict else None

df['billing_type_id'] = df['billing_type'].map(billing_type_id)

del df['billing_type']

In [207]:
def employment_id(x):
    return x['id'] if type(x)==dict else None

df['employment_id'] = df['employment'].map(employment_id)
del df['employment']

In [208]:
def schedule_id(x):
    return x['id'] if type(x)==dict else None

df['schedule_id'] = df['schedule'].map(schedule_id)
del df['schedule']

In [209]:
def experience_id(x):
    return x['id'] if type(x)==dict else None

df['experience_id'] = df['experience'].map(schedule_id)
del df['experience']

In [210]:
df['all_text'] = [
    (x if type(x)==str else ' ') + ' ' +
    (y if type(y)==str else ' ') 
    for x, y in zip(
    df['name'],
    df['description'])]

#del df['name']
#del df['description']

In [211]:
from bs4 import BeautifulSoup

In [212]:
df['all_text'].head()

0    Карщик-грузчик <strong>Обязанности:</strong> <...
1                                            Not Found
2    Менеджер по персоналу в офис <strong>Обязаннос...
3    Механик энергоблока <p>Крупнейший международны...
4    Менеджер по продажам автомобилей <p><strong>Се...
Name: all_text, dtype: object

In [213]:
df['all_text'] = df['all_text'].map(
    lambda x: BeautifulSoup(x, "lxml").text
)

In [231]:
del df['accept_handicapped']
del df['accept_kids']
del df['alternate_url']

del df['apply_alternate_url']
del df['archived']
del df['address']
del df['contacts']
del df['employer']
del df['driver_license_types']
del df['id']
del df['errors']
del df['negotiations_url']
del df['relations']
del df['site']
del df['key_skills']
del df['test']
del df['suitable_resumes_url']
del df['name']
del df['description']
del df['branded_description']
del df['department']
del df['code']

In [232]:
df.to_csv('headhunter.csv', sep=',' )